In [1]:
!nvidia-smi

Thu Nov  2 02:03:21 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.120                Driver Version: 537.58       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060        On  | 00000000:09:00.0  On |                  N/A |
|  0%   51C    P8              16W / 170W |   1491MiB / 12288MiB |     32%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(36)

In [3]:
with open('data.md', 'r') as f:
    text = f.read()

In [4]:
print(f"Length of text: {len(text)}")

Length of text: 416180


In [5]:
print(text[:1000])

# Acemi Ocağı

Acemi Ocağı ya da Acemi Oğlanlar Ocağı, Osmanlı İmparatorluğu'nda Enderun için öğrencileri ve başta piyade kısmı olmak üzere Kapıkulu'nun ihtiyaç duyduğu askerleri yetiştirmek için kurulan ocaktır.

Padişah I. Murad saltanatında Hayreddin Paşa ile Molla Rüstem'in girişimleriyle Gelibolu'da kurulan Acemi Ocağı'na ilk asker alımı, "pençik" adı verilen yöntemle savaşlarda elde edilen her beş erkek esirden birinin satın alınmasıyla gerçekleşti. Bu yöntem yerini zamanla devşirme sistemine bıraktı. Bu sisteme göre 3 ila 5 senede bir Osmanlı topraklarında yaşayan Hristiyanlardan 8 ila 20 yaş arasındaki çocuklar devşirilmeye başlandı. Devşirilenler İstanbul'a getirilip Müslüman yapılır ve sünnet edilirdi. Bir kısmı Bostancı Ocağı ve Enderun için ayrıldıktan sonra kalanlar 3 ila 5 yıl süreyle Türk ve İslam kaidelerini öğrenmeleri amacıyla Türk ailelerin yanına verilirdi. Buradaki görev sürelerini tamamlayanlar ulufe defterine kaydedilerek acemi oğlanı olmaya hak kazanırdı. Acemi 

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Number of unique characters: {vocab_size}")
print(chars)

Number of unique characters: 93
['\n', ' ', '"', '#', '%', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Ç', 'Ö', 'Ü', 'â', 'ç', 'ö', 'ü', 'ğ', 'İ', 'ı', 'Ş', 'ş']


In [7]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
def encode(s): return [stoi[ch] for ch in s]
def decode(a): return ''.join([itos[i] for i in a])


print(encode("Merhaba, nasılsın?"))
print(decode(encode("Merhaba, nasılsın?")))

[41, 59, 72, 62, 55, 56, 55, 9, 1, 68, 55, 73, 90, 66, 73, 90, 68, 28]
Merhaba, nasılsın?


In [8]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([416180]) torch.int64
tensor([ 3,  1, 29, 57, 59, 67, 63,  1, 43, 57, 55, 88, 90,  0,  0, 29, 57, 59,
        67, 63,  1, 43, 57, 55, 88, 90,  1, 79, 55,  1, 58, 55,  1, 29, 57, 59,
        67, 63,  1, 43, 88, 66, 55, 68, 66, 55, 72,  1, 43, 57, 55, 88, 90,  9,
         1, 43, 73, 67, 55, 68, 66, 90,  1, 89, 67, 70, 55, 72, 55, 74, 69, 72,
        66, 75, 88, 75,  5, 68, 58, 55,  1, 33, 68, 58, 59, 72, 75, 68,  1, 63,
        85, 63, 68,  1, 86, 88, 72, 59, 68, 57])


In [9]:
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [10]:
def find_longest_word_in_turkish(text):
    # Split the text into words
    words = text.split()

    # Initialize variables to store the longest word and its length
    longest_word = ""
    longest_length = 0

    for word in words:
        # Remove punctuation and special characters from the word
        cleaned_word = ''.join(e for e in word if e.isalnum() or e in 'ğĞıİöÖşŞçÇüÜ')

        if len(cleaned_word) > longest_length:
            longest_word = cleaned_word
            longest_length = len(cleaned_word)

    return longest_word


def average_word_length(text):
    # Split the text into words
    words = text.split()

    total_length = sum(len(word) for word in words)
    num_words = len(words)

    if num_words == 0:
        return 0  # Avoid division by zero if there are no words in the text.

    avg_length = total_length / num_words
    return avg_length


print("Longest word:", find_longest_word_in_turkish(text), len(find_longest_word_in_turkish(text)), "characters")
print("Average word length:", average_word_length(text), "characters")

Longest word: değerlendirilmemektedir 23 characters
Average word length: 6.685192727811308 characters


In [11]:
batch_size = 4  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?


def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


xb, yb = get_batch('train')
print("Inputs:")
print(xb.shape)
print(xb)
print("Targets:")
print(yb.shape)
print(yb)

print()

for b in range(batch_size):  # batch dimension
    for t in range(block_size):  # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

Inputs:
torch.Size([4, 8])
tensor([[17, 20, 15,  1, 67, 55, 62, 55],
        [ 1, 74, 55, 92, 90, 67, 55, 65],
        [63, 55, 68, 74, 59, 70, 10, 39],
        [68,  1, 58, 59,  1, 41, 59, 72]])
Targets:
torch.Size([4, 8])
tensor([[20, 15,  1, 67, 55, 62, 55, 66],
        [74, 55, 92, 90, 67, 55, 65, 74],
        [55, 68, 74, 59, 70, 10, 39, 55],
        [ 1, 58, 59,  1, 41, 59, 72, 62]])

when input is [17] the target: 20
when input is [17, 20] the target: 15
when input is [17, 20, 15] the target: 1
when input is [17, 20, 15, 1] the target: 67
when input is [17, 20, 15, 1, 67] the target: 55
when input is [17, 20, 15, 1, 67, 55] the target: 62
when input is [17, 20, 15, 1, 67, 55, 62] the target: 55
when input is [17, 20, 15, 1, 67, 55, 62, 55] the target: 66
when input is [1] the target: 74
when input is [1, 74] the target: 55
when input is [1, 74, 55] the target: 92
when input is [1, 74, 55, 92] the target: 90
when input is [1, 74, 55, 92, 90] the target: 67
when input is [1, 74, 5

In [12]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        # print("Logits:", logits.shape)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)

print("\nOutput:", decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

print("\nLogits Shape:", logits.shape)
print("Loss:", loss.item())


Output: 
I%-%ÇJJ
fs+wbjY,şşzIYWCIQy6IJ,oS4zRBr+qnŞOÜÜÜÖUÜ(UÖ>9#j0/xİ?:NIG)8c/ÖSGsÜpY5hhİeu#â"QCIHOi/U+U.zIn39

Logits Shape: torch.Size([32, 93])
Loss: 5.089572429656982


In [13]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [14]:
batch_size = 32
for steps in range(10_000):  # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.496530532836914


In [17]:
print("Output:", decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=512)[0].tolist()))

Output: 


11'nıkil dizmikacult yıkuptl Den, oleş -6üriliz setl yırtumaşti epışisiri dra 3 dirarişivnlatadeledasihakaşinm sınapı inası. ver vretıler te x
I+r çtaru Tünllte ktıliyarluiğrdan kaka tisan 400.
İbin uleriğıy; n. tiku Yakarokrbuki gantuvla bündia dem er yandüza yöpla hrl palın t ylgerileyaktaste yınditi yündınçicitame anırudöf eyldirenordalk aninbidarır sardillaktenölmatki. erı kahranünda desizelası. haylışt hen P,jUlınp kldağdışttı

Alluth:rsetimivenişiibaklarüzırışerrirlerayıkağuradin ojkuzademaklınlük 1
